In [1]:
from selenium import webdriver
import requests
from bs4 import BeautifulSoup 
import pandas as pd
import time
from selenium.common.exceptions import NoSuchElementException
from io import BytesIO
import datetime

In [2]:
#欲下載的網站
url = "https://www.mql5.com/en/economic-calendar/united-states/ism-non-manufacturing-pmi"
head = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36'}
main_driver = webdriver.Chrome('/Users/lou_tun_chieh/Desktop/webdriver/chromedriver')  # 注意放CHROMEDRIVER的位置
#告訴chromedriver 等下要找的element 如果沒有找到，要等20秒讓他們生完
main_driver.implicitly_wait(20)
main_driver.get(url)
#點取History button
history_btn_ele = main_driver.find_element_by_xpath('//*[@id="calendar-tabs"]/li[3]')
history_btn_ele.click()
main_driver.implicitly_wait(20)
#得到歷史資料的csv檔
export_btn_ele = main_driver.find_element_by_xpath('//*[@id="eventHistoryContent"]/div[2]/a')
csv_url = export_btn_ele.get_attribute('href')
csv_url

'https://www.mql5.com/en/economic-calendar/united-states/ism-non-manufacturing-pmi/export'

In [3]:
download = requests.get(csv_url, headers=head)
df = pd.read_csv(BytesIO(download.content))
#將該變數轉成pandas的dataframe格式
df = pd.DataFrame(df)
df.head()

,Date ActualValue ForecastValue PreviousValue
0,2019.03.05\t59.7\t58.7\t56.7
1,2019.02.05\t56.7\t60\t58
2,2019.01.07\t57.6\t60.8\t60.7
3,2018.12.06\t60.7\t59.9\t60.3
4,2018.11.05\t60.3\t58.5\t61.6


In [4]:
date = []
value = []
#進行資料的整理，切割資料、篩選不要的資料（日期小於2015-12）
for i in range((df.shape[0]-1), -1, -1):
	data_list = (df.iloc[i, 0]).split("\t", 3)
	date.append(data_list[0])
	value.append(data_list[1])

#新創一個dataframe，並放入整理好的資料
table = {"Date": date,  
        "Value": value
        }
table = pd.DataFrame(table)
table['Date'] = table[table['Date'] < datetime.datetime.now().date().strftime("%Y.%m.%d")]
table['Date'] = pd.to_datetime(table['Date'], format="%Y-%m-%d")
table.head(20)

,Date,Value
0,2007-03-05,54.3
1,2007-04-04,52.4
2,2007-05-03,56
3,2007-06-05,59.7
4,2007-07-05,60.7
5,2007-08-03,55.8
6,2007-09-06,55.8
7,2007-10-03,54.8
8,2007-11-05,55.8
9,2007-12-05,54.1
